## Import Libraries

In [73]:
import os
import sys

## Read text from PDF
##### Provide the path of PDF file before initialized everything

In [74]:
data='data.pdf'

In [75]:
data

'data.pdf'

## Extract data by reading

In [76]:
import PyPDF2
from docx import Document

In [77]:
def get_pdf_text(pdf_docs):
    text =""
    for pdf in pdf_docs:
        pdf_reader =PyPDF2.PdfReader(pdf_docs) 
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

'''The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'''


'The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'

In [78]:
def extract(doc):
    _, file_extension = doc.split('.')
    print(file_extension)
    if file_extension == 'pdf':
        return get_pdf_text(doc)
    elif file_extension == 'docx':
        return get_pdf_text(doc)
    else:
        raise ValueError("Unsupported file format")

In [79]:
text_data=extract(data)


pdf


In [80]:
print(text_data)

 
 
University of Ruhuna 
Faculty of Science 
Matara, Sri Lanka  
 
Student  Handbook  - 2020  
 
 
 
Bachelor  of Science 
And  
Bachelor of Computer Science 
(General and Honours) Degree 
Programmes  
 
2019/2020
2   
 
Department  of Computer  Science  53 
1.1 Research  Areas  ................................ ................................ ................................ ...............  53 
1.2 Head  of the Department  ................................ ................................ ................................  54 
1.3 Members  of Academic  Staff ................................ ................................ ...........................  54 
1.4 Members  of the Academic  Support  Staff ................................ ................................ ..... 55 
1.5 Course  Units  in Computer  Science  for B.Sc.  (General)  Degree  ..............................  55 
1.6 ICT Course  units  of Bachelor  of Science  (General)  Degree  ................................ ..... 5

### Convert each text into chunks
##### need to split the text using CharacterTextSplitter (which is basically present in Langchain.text_splitter) such that it should not increase token size


In [81]:
from langchain.text_splitter import CharacterTextSplitter

In [82]:
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size =1300,
        chunk_overlap = 300,
        length_function =len,

    )

In [83]:
chunks = text_splitter.split_text(text_data)

#taking the particular text & splitting based on information

In [84]:
len(chunks)

481

In [85]:
print(chunks[12])

cal hrs.)  Introduction  to System  Development  environment,  The Object  paradigm,  Objec - 
tory  S/W  development  process,  Object  Oriented  Modelling  using  UML,  Use case  diagrams, 
Class  diagram,  Interaction  diagram, Class  declaration, Method  implementation, Informa - 
tion hiding and Access modifiers, Class Hierarchy (inheritance), Polymorphism, Packages 
and Data  abstraction.  Practical  using  Rational  Rose  and OO Programming  using  Java/C++ 
Evaluatio n Method:  Practical  Examination  30%  and End Semester  Examina - 
tion 70% (Student must obtain at least 35% from the theory paper and the 30% 
from the practical paper)  7  COM213 α: Operating  Systems  (15 lecture  hrs. & 22.5  practical  hrs.)   Introduc - 
tion, Process Management, CPU Scheduling, Deadlocks, Memory Management, File -System 
Implementation, I/O Systems.  Practical oriented  
Evaluation Method:  Practical Examination 30% and End  Semester Examina - 
tion 70% (Student must obtain at least 35% f

In [86]:
print(chunks)

['University of Ruhuna \nFaculty of Science \nMatara, Sri Lanka  \n \nStudent  Handbook  - 2020  \n \n \n \nBachelor  of Science \nAnd  \nBachelor of Computer Science \n(General and Honours) Degree \nProgrammes  \n \n2019/2020\n2   \n \nDepartment  of Computer  Science  53 \n1.1 Research  Areas  ................................ ................................ ................................ ...............  53 \n1.2 Head  of the Department  ................................ ................................ ................................  54 \n1.3 Members  of Academic  Staff ................................ ................................ ...........................  54 \n1.4 Members  of the Academic  Support  Staff ................................ ................................ ..... 55 \n1.5 Course  Units  in Computer  Science  for B.Sc.  (General)  Degree  ..............................  55 \n1.6 ICT Course  units  of Bachelor  of Science  (General)  Degree  ...................

## Convert chunks into vectors(Text Embeddings)

In [89]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


ModuleNotFoundError: No module named 'langchain_google_genai'

In [ ]:
def get_vector_store(text_chunks):